Project

In [21]:
#Library import and basic functions
import psycopg2
import csv
import psycopg2.extras
import re
import shapefile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb; sb.set()
import math

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = 'zhua2542'#'<your_unikey>'
    YOUR_PW     = '480537405'#'<your_SID>'
    try: 
        conn = psycopg2.connect(host='soit-db-pro-1.ucc.usyd.edu.au',
                                database='y19s1d2x01_'+YOUR_UNIKEY,
                                user='y19s1d2x01_'+YOUR_UNIKEY,
                                password=YOUR_PW) 
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return conn


def pgexec(conn, sqlcmd, args, msg, silent=False):
    retval = False
    with conn:
        with conn.cursor() as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if silent == False:
                    print("success: " + msg)
                retval = True
            except Exception as e:
                if silent == False:
                    print("db error: ")
                    print(e)
    return retval


def pgquery(conn, sqlcmd, args, silent=False):
    """ utility function to execute some SQL query statement
        can take optional arguments to fill in (dictionary)
        will print out on screen the result set of the query
        error and transaction handling built-in """
    retval = False
    result = []
    with conn:
        with conn.cursor() as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if silent == False:
                    for record in cur:
                        result.append(record)
                retval = True
            except Exception as e:
                if silent == False:
                    print("db read error: ")
                    print(e)
    return result

def clean_empty_string(data):
    for row in data:
        for key, value in row.items():
            if value == "":
                row[key] = None


def create_table(file, queries):
    data = list(csv.DictReader(open(file + '.csv')))
    clean_empty_string(data)
    # to reset table
    pgexec(conn, "DROP TABLE IF EXISTS " + file, None, "Reset Table " + file)
    # create table using schema
    pgexec(conn, queries[0], None, "Create Table " + file)
    # insert values to table
    for row in data:
        pgexec(conn, queries[1], row, "row inserted")
        
def create_column(conn, col_name, table_name, type):
    query = """ALTER TABLE {}
                DROP COLUMN IF EXISTS {}, 
                ADD COLUMN {} {};""".format(table_name, col_name, col_name, type)

    pgexec(conn, query, None, "Created Column " + col_name + " on " + table_name)

def update_column_with_another(conn, col_name, table_name, value):
    query = """UPDATE {}
               SET {} = COALESCE{}""".format(table_name, col_name, value)

    pgexec(conn, query, None, "Update " + col_name + " on " + table_name +" with " + value)

def fix_NULL(item):
    if (item == None):
        return 0
    
    return item


def update_service_balance(conn):
    subquery = """SELECT * FROM neighbourhoods JOIN businessstats USING (area_id);"""
    result = pgquery(conn, subquery, None)

    for row in result:

        recreation = row[-1]
        education = row[-2]
        health = row[-3]
        food = row[-4]
        retail = row[-5]

        recreation = fix_NULL(recreation)
        education = fix_NULL(education)
        food = fix_NULL(food)
        retail = fix_NULL(retail)
        health = fix_NULL(health)
        
        sum = recreation + education + health + food + retail
        if (sum == 0):
            continue
        
        service_balance = (education * 5 + food * 4 + retail * 3 + recreation * 2 + health) / sum

        query = """UPDATE neighbourhoods SET service_balance = {} WHERE area_id = {}""".format(service_balance, row[0])
        pgexec(conn, query, None, "Set service balance of area " + str(row[0]) + " to " + str(service_balance))

In [ ]:
#SQL table set up
conn = pgconnect()

    # =====Queries for creating and inserting table=====
business_stats_schema = """CREATE TABLE IF NOT EXISTS BusinessStats (
                                        area_id INT NOT NULL PRIMARY KEY,
                                        num_businesses INT,
                                        retail_trade INT,
                                        accommodation_and_food_services INT,
                                        health_care_and_social_assistance INT,
                                        education_and_training INT,
                                        arts_and_recreation_services INT
                               )"""
business_stats_insert_stmt = """INSERT INTO BusinessStats(
                                            area_id,
                                            num_businesses,
                                            retail_trade,
                                            accommodation_and_food_services,
                                            health_care_and_social_assistance,
                                            education_and_training,
                                            arts_and_recreation_services)
                                            VALUES (
                                            %(area_id)s,
                                            %(num_businesses)s,
                                            %(retail_trade)s,
                                            %(accommodation_and_food_services)s,
                                            %(health_care_and_social_assistance)s,
                                            %(education_and_training)s,
                                            %(arts_and_recreation_services)s)"""

bike_pods_schema = """CREATE TABLE IF NOT EXISTS BikeSharingPods(
                            station_id INT NOT NULL PRIMARY KEY,
                            name VARCHAR(70),
                            num_bikes INT,
                            num_scooters INT,
                            latitude FLOAT,
                            longitude FLOAT,
                            description VARCHAR(500)
                        )"""
bike_pods_insert_stmt = """INSERT INTO BikeSharingPods(
                                station_id,
                                name,
                                num_bikes,
                                num_scooters,
                                latitude,
                                longitude,
                                description)
                                VALUES (
                                %(station_id)s,
                                %(name)s,
                                %(num_bikes)s,
                                %(num_scooters)s,
                                %(latitude)s,
                                %(longitude)s,
                                %(description)s)"""

census_stats_schema = """CREATE TABLE IF NOT EXISTS CensusStats(
                                area_id INT NOT NULL PRIMARY KEY,
                                median_annual_household_income INT,
                                avg_monthly_rent INT
                            )"""
census_stats_insert_stmt = """INSERT INTO CensusStats(
                                    area_id,
                                    median_annual_household_income,
                                    avg_monthly_rent)
                                    VALUES (
                                    %(area_id)s,
                                    %(median_annual_household_income)s,
                                    %(avg_monthly_rent)s)"""

neighbourhoods_schema = """CREATE TABLE IF NOT EXISTS Neighbourhoods(
                                area_id INT NOT NULL PRIMARY KEY,
                                area_name VARCHAR(70),
                                land_area FLOAT,
                                population INT,
                                number_of_dwellings INT,
                                number_of_businesses INT
                            )"""
neighbourhoods_insert_stmt = """INSERT INTO Neighbourhoods(
                                    area_id,
                                    area_name,
                                    land_area,
                                    population,
                                    number_of_dwellings,
                                    number_of_businesses)
                                    VALUES (
                                    %(area_id)s,
                                    %(area_name)s,
                                    %(land_area)s,
                                    %(population)s,
                                    %(number_of_dwellings)s,
                                    %(number_of_businesses)s)"""

statistical_areas_schema = """CREATE TABLE IF NOT EXISTS StatisticalAreas(
                                    area_id INT NOT NULL PRIMARY KEY,
                                    area_name VARCHAR(70),
                                    parent_area_id INT
                                )"""
statistical_areas_insert_stmt = """INSERT INTO StatisticalAreas(
                                        area_id,
                                        area_name,
                                        parent_area_id)
                                        VALUES (
                                        %(area_id)s,
                                        %(area_name)s,
                                        %(parent_area_id)s)"""
    # =====QUERIES END=====

    # Queries stored in key value pair, with file name as key, and queries stored as lists
queries = {
        'BusinessStats': [business_stats_schema, business_stats_insert_stmt],
        'BikeSharingPods': [bike_pods_schema, bike_pods_insert_stmt],
        'CensusStats': [census_stats_schema, census_stats_insert_stmt],
        'Neighbourhoods': [neighbourhoods_schema, neighbourhoods_insert_stmt],
        'StatisticalAreas': [statistical_areas_schema, statistical_areas_insert_stmt]
    }

    # Loop through each file and queries to create a table
for k, v in queries.items():
    create_table(k, v)
    

In [ ]:
#Add score related columns and populate
create_column(conn, "population_density", "neighbourhoods", "DOUBLE PRECISION")
create_column(conn, "dwelling_density", "neighbourhoods", "DOUBLE PRECISION")
create_column(conn, "service_balance", "neighbourhoods", "DOUBLE PRECISION")
create_column(conn, "bikepod_density", "neighbourhoods", "DOUBLE PRECISION")
update_column_with_another(conn, "population_density", "neighbourhoods", "(population / land_area)")
update_column_with_another(conn, "dwelling_density", "neighbourhoods", "(number_of_dwellings / land_area)")

update_service_balance(conn)

In [ ]:
#Spatial join bikepods and neighbourhoods

sf = shapefile.Reader("1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp", encoding="iso-8859-1")
# which shpe type is it?
print(sf)

print(sf.fields)

for i in range(0, 10):
    print(sf.record(i))

create_column(conn, "geom", "neighbourhoods", "GEOMETRY(Polygon, 4326)")

update_stmt = """UPDATE neighbourhoods SET geom = ST_GEOMFROMTEXT(%(geom)s, 4326) WHERE area_id = %(area_id)s;"""

shapes = sf.shapes()
records= sf.records()


areas = pgquery(conn, "SELECT area_id FROM neighbourhoods;", "Find existing areas")

area_ids = []
for r in areas:
    area_ids.append(r[0])


row = {}
for i in range(0, len(shapes)):
    record = sf.record(i)

    if int(record[0]) in area_ids:
        shape  = sf.shape(i)

        row['area_id']=record[0]
        
        # prepare the polygon data
        # this is a bit complex with our dataset as it has complex polygons, some with multiple parts...
        row['geom']="POLYGON(("
        i=0
        for x, y in shape.points:
            row['geom']+="%s %s," % (x,y)
            # check for start of a new polygon part
            i += 1
            if i in shape.parts:
                row['geom']= re.sub(",$", "),(", row['geom'])
        # properly end the polygon string
        row['geom'] = re.sub(",$", "))", row['geom'])
        
        # finally: insert new row into the table
        pgexec(conn, update_stmt, args=row, msg="inserted "+str(record[2]))

index_command = "CREATE INDEX neighbourhood_idx ON neighbourhoods USING GIST (geom);"
pgexec(conn, index_command, None, "Created index")

In [ ]:
#compute z score

create_column(conn, "cyclability_score", "neighbourhoods", "DOUBLE PRECISION")

avg_pd = pgquery(conn, "SELECT AVG(population_density) FROM neighbourhoods;", None)
avg_dd = pgquery(conn, "SELECT AVG(dwelling_density) FROM neighbourhoods;", None)
avg_sb = pgquery(conn, "SELECT AVG(service_balance) FROM neighbourhoods;", None)
avg_bd = pgquery(conn, "SELECT AVG(bikepod_density) FROM neighbourhoods;", None)

std_pd = pgquery(conn, "SELECT STDDEV(population_density) FROM neighbourhoods;", None)
std_dd = pgquery(conn, "SELECT STDDEV(dwelling_density) FROM neighbourhoods;", None)
std_sb = pgquery(conn, "SELECT STDDEV(service_balance) FROM neighbourhoods;", None)
std_bd = pgquery(conn, "SELECT STDDEV(bikepod_density) FROM neighbourhoods;", None)

avg_pd = float(avg_pd[0][0])
avg_dd = float(avg_dd[0][0])
avg_sb = float(avg_sb[0][0])
avg_bd = float(avg_bd[0][0])

std_pd = float(std_pd[0][0])
std_dd = float(std_dd[0][0])
std_sb = float(std_sb[0][0])
std_bd = float(std_bd[0][0])

print(avg_pd)
result = pgquery(conn, "SELECT * FROM neighbourhoods", None)

def additional_score():
    return 0

for row in result:
    z_score = (float(fix_NULL(row[-3])) - avg_bd) / std_bd
    z_score += (float(fix_NULL(row[-4])) - avg_sb) / std_sb
    z_score += (float(fix_NULL(row[-5])) - avg_dd) / std_dd
    z_score += (float(fix_NULL(row[-6])) - avg_pd) / std_pd
    z_score += additional_score()

    query = "UPDATE neighbourhoods SET cyclability_score = {} WHERE area_id = {}".format(z_score, row[0])
    pgexec(conn, query, None, "Updating score for {}".format(row[1]))

In [ ]:
result = pgquery(conn, "SELECT * FROM neighbourhoods JOIN censusstats USING (area_id) ORDER BY area_id;", None)
z_score = []
income = []
rent = []
for row in result:
    z_score.append(row[-3])
    income.append(row[-2])
    rent.append(row[-1])

score_income = pd.DataFrame({"score": z_score, "income": income})
score_rent = pd.DataFrame({"score": z_score, "rent": rent})

plt.figure()
plot1 = sb.regplot(x = "score", y= "income", data = score_income, fit_reg = False)
fig1 = plot1.get_figure()
fig1.savefig("score_income")

plt.figure()
plot2 = sb.regplot(x = "score", y = "rent", data = score_rent, fit_reg = False)
fig2 = plot2.get_figure()
fig2.savefig("score_rent")